In [ ]:
import requests as r
from bs4 import BeautifulSoup
import pandas as pd

In [ ]:
def get_soup_content(url):
    resp=r.get(url).content
    BeautifulSoup(resp)
    return soup   

In [ ]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException

def get_soup_from_selenium(url):
    driver= webdriver.Chrome('chromedriver_win32/chromedriver')
    driver.get(url)
    delay = 50 # seconds
    try:
        WebDriverWait(driver, delay).until(EC.presence_of_element_located((By.ID, 'search_location_state')))
        print ("Page is ready!")
        soup = BeautifulSoup(driver.page_source,"lxml")
    except TimeoutException:
        print ("Loading took too much time!")
    return soup
    
get_soup_from_selenium('https://www.startupranking.com/top/0/2')

In [ ]:
def scrap_home_page(url):
    print(url)
    soup=get_soup_from_selenium(url)
    tabs=soup.find_all('table',{'class':'rank_table'})[0]
    rows=tabs.find_all('tr')
    list_urls=[i.select('a')[0].get('href') for i in rows]
    list_urls.pop(0)
    return list_urls

scrap_home_page('https://www.startupranking.com/top')

In [ ]:
def scrap_startup_page(url_startup,total_funding_amount):

    resp2=r.get(url_startup).content
    
    soup2=BeautifulSoup(resp2)
    country_name = soup2.select('img.flag')[0].get('alt')
    name=soup2.select('div.su-info h2 a')[0].text
    # founded date
    tabb = soup2.select('div.su-info p.su-loc')
    if len(tabb)==0: 
        founded_date = ''
    else:
        founded_date =  tabb[0].text.strip()
    
    cat = soup2.select('div.su-tags.group ul li')
    category_list =[i.text for i in cat]
    category_string = ','.join(category_list)
    dict_info={'name':[name],'country_name':[country_name],'founded_date':[founded_date],'category_list': [category_string],'total_funding_amout':[total_funding_amount]}
    df=pd.DataFrame(dict_info)
    return df



scrap_startup_page('https://www.startupranking.com/ifttt',210100000)

In [ ]:
def get_url_funding_page(url_startup_page):
    resp=r.get(url_startup_page).content
    soup=BeautifulSoup(resp)
    tabb = soup.select('.small-table-container')[1].select('a.see-all')
    if len(tabb)==0: 
        return ''
    else:
        return tabb[0].get('href')
get_url_funding_page('https://www.startupranking.com/ifttt')

In [ ]:
import re
def get_total_funding_amount(url_funding_page):
    if url_funding_page=='':
        return 0
    else:
        resp=r.get(url_funding_page).content
        soup=BeautifulSoup(resp)
        tabs2 = soup.select('.ranks')[0]
        rows2=tabs2.find_all('tr')
        list_funding_amount = [i.select('span')[0].text for i in rows2]
        list_funding_amount = list(filter(lambda a: a != 'Undisclosed amount', list_funding_amount))
        funding_amount_dolar = [int(re.search(r'[0-9]+',i.replace(',','')).group(0)) for i in list_funding_amount]
        Total_funding_amount = sum(funding_amount_dolar)
        return Total_funding_amount

get_total_funding_amount('https://www.startupranking.com/startup/ifttt/funding-rounds')

In [ ]:
import pymysql
data = scrap_startup_page('https://www.startupranking.com/500px',0,210100000)
def save_to_sql(data):
    connection = pymysql.connect(host='localhost',
                             user='root',
                             password='safa',
                             db='startupdb')
    cursor = connection.cursor()
    # creating column list for insertion
    cols = "`,`".join([str(i) for i in data.columns.tolist()])

    # Insert DataFrame recrds one by one.
    for i,row in data.iterrows():
        sql = "INSERT INTO `startup` (`" +cols + "`) VALUES (" + "%s,"*(len(row)-1) + "%s)"
        cursor.execute(sql, tuple(row))

        # the connection is not autocommitted by default, so we must commit to save our changes
        connection.commit()
save_to_sql(data)

In [ ]:
import time


def scrap_all_startup_perpage(url_home,list_urls):
    dataframe = pd.DataFrame()
    i=0
    for url_startup in list_urls:
        i+=1
        whole_url_startup = url_home + url_startup
        print('url =', whole_url_startup)
        
        url_funding = get_url_funding_page(whole_url_startup)
        if url_funding=='':
            whole_url_funding = ''
        else:
            whole_url_funding = url_home + url_funding
       # dataframe['total_funding_amout'] = get_total_funding_amount(whole_url_funding)
        total = get_total_funding_amount(whole_url_funding)
        new_df = scrap_startup_page(whole_url_startup,i,total)
        dataframe = dataframe.append(new_df)
        time.sleep(1)
        if i%10==0:
            pos = i-10
            print(f'dataframe.iloc[{pos},{i}]')
            df = dataframe.iloc[pos:i]
            save_to_sql(df)
    
    return dataframe

url = 'https://www.startupranking.com/top'
url_home = 'https://www.startupranking.com'
list_urls = scrap_home_page(url)
dataframe = scrap_all_startup_perpage(url_home,list_urls)

In [ ]:
dataframe

In [ ]:
def iterate_startup_page(n,url_home,url_top):
    dataframe = pd.DataFrame()
    for i in range(int(n)):
        url_new = url_top+'/'+str(i+1)
        list_urls = scrap_home_page(url_new)
        df = scrap_all_startup_perpage(url_home,list_urls)
        dataframe = dataframe.append(df)     
    dataframe.to_csv('startup2.csv')
    return dataframe

dataframe2 = iterate_startup_page('2','https://www.startupranking.com','https://www.startupranking.com/top/0')

In [ ]:
dataframe2